In [ ]:
import urllib.request
from bs4 import BeautifulSoup

url = 'http://fund.eastmoney.com/allfund.html'
html = urllib.request.urlopen(url)
html = html.read()

soup = BeautifulSoup(html, 'lxml')#如果使用lxml解析不完全请使用html5lib

In [ ]:
soup.select("#code_content")[0]

In [ ]:
fund_codes = []
for a in soup.select("#code_content")[0].find_all("a"):
    try:
        if('http://fund.eastmoney.com/' in a['href']):
            fund_codes.append(a['href'].split("/")[-1].split(".")[0])
    except:
        continue

In [ ]:
fund_codes

In [ ]:
import time
html2 = urllib.request.urlopen('http://fundf10.eastmoney.com/FundArchivesDatas.aspx?type=jjcc&code='+fund_codes[0]+'&topline=500&year='+time.strftime("%Y",time.localtime(time.time())))
html2 = html2.read()
soup2 = BeautifulSoup(html2, 'lxml')

In [ ]:
soup2.select(".box")[0].find_all("h4")[0].select("label")[0].a.text

In [ ]:
soup2.select(".box")[0].find_all("h4")[0].select("label")[1].font.text

In [ ]:
tmp = soup2.select(".box")[0].find_all("tbody")[0].find_all("tr")[0].select("td")

In [ ]:
tmp

In [ ]:
len(tmp)

In [ ]:
#股票代码
tmp[1].text

In [ ]:
#股票名称
tmp[2].text

In [ ]:
#占净值比例
tmp[6].text

In [ ]:
#持股数（万股）
tmp[7].text

In [ ]:
#持仓市值（万元）
tmp[8].text

# 预处理实验完毕，接下来批处理

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import time

url = 'http://fund.eastmoney.com/allfund.html'
html = urllib.request.urlopen(url)
html = html.read()

soup = BeautifulSoup(html, 'lxml')#如果使用lxml解析不完全请使用html5lib

fund_codes = []
for a in soup.select("#code_content")[0].find_all("a"):
    try:
        if('http://fund.eastmoney.com/' in a['href']):
            fund_codes.append(a['href'].split("/")[-1].split(".")[0])
    except:
        continue

fund_stocks = {'fund_code':[], 'fund_name' : [], 'deadline': [],\
                          'stock_code':[],'stock_name':[],'position_ratio':[],\
                          'number_shares':[],'stocks_value':[]}

for code in fund_codes:
    html2 = urllib.request.urlopen('http://fundf10.eastmoney.com/FundArchivesDatas.aspx?type=jjcc&code='+code+'&topline=500&year='+time.strftime("%Y",time.localtime(time.time())))
    html2 = html2.read()
    soup2 = BeautifulSoup(html2, 'lxml')

    for box in soup2.select(".box"):
        fund_name = box.find_all("h4")[0].select("label")[0].a.text
        deadline = box.find_all("h4")[0].select("label")[1].font.text
        for stocks in box.find_all("tbody"):
            for stock in stocks.find_all("tr"):
                tmp = stock.select("td")
                stock_code = tmp[1].text
                stock_name = tmp[2].text
                tmp2 = stock.select(".tor")
                if(len(tmp2) == 0):
                    tmp2 = stock.select(".toc")
                    if(len(tmp2) > 5):
                        position_ratio = tmp2[4].text
                        number_shares = tmp2[5].text
                        stocks_value = tmp2[6].text
                    else:
                        position_ratio = tmp2[2].text
                        number_shares = tmp2[3].text
                        stocks_value = tmp2[4].text
                else:
                    if(len(tmp2) > 3):
                        position_ratio = tmp2[2].text
                        number_shares = tmp2[3].text
                        stocks_value = tmp2[4].text
                    else:
                        position_ratio = tmp2[0].text
                        number_shares = tmp2[1].text
                        stocks_value = tmp2[2].text
                fund_stocks['fund_code'].append(code)
                fund_stocks['fund_name'].append(fund_name)
                fund_stocks['deadline'].append(deadline)
                fund_stocks['stock_code'].append(stock_code)
                fund_stocks['stock_name'].append(stock_name)
                fund_stocks['position_ratio'].append(position_ratio)
                fund_stocks['number_shares'].append(number_shares)
                fund_stocks['stocks_value'].append(stocks_value)

In [ ]:
import pandas as pd
df = pd.DataFrame(fund_stocks)

In [ ]:
df

In [ ]:
df.to_csv("fund_stocks.csv",index=False)

In [ ]:
len(df['fund_name'].unique())

## 股票重仓基金数

In [ ]:
stock_fund_num_latest = df[df["deadline"] == '2020-09-30'][['fund_name','stock_name']].groupby(['stock_name']).count().sort_values(by='fund_name',ascending=False)
stock_fund_num_latest

## 上个季度持仓数

In [ ]:
stock_fund_num_last = df[df["deadline"] == '2020-06-30'][['fund_name','stock_name']].groupby(['stock_name']).count().sort_values(by='fund_name',ascending=False)
stock_fund_num_last

## 与上个季度建仓基金数比较

In [ ]:
stock_fund_num_latest = stock_fund_num_latest.reset_index()
stock_fund_num_latest.columns = ['stock_name','fund_num_2020-09-30']
stock_fund_num_last = stock_fund_num_last.reset_index()
stock_fund_num_last.columns = ['stock_name', 'fund_num_2020_06_30']

In [ ]:
stock_fund_compare = pd.merge(stock_fund_num_last,stock_fund_num_latest, on = 'stock_name')
stock_fund_compare['increase'] = stock_fund_compare['fund_num_2020-09-30'] - stock_fund_compare['fund_num_2020_06_30']
stock_fund_compare.sort_values(by="increase",ascending=False)

In [ ]:
stock_fund_compare[stock_fund_compare['stock_name'] == '隆基股份']

## 基金总持仓股数比较（万股）